# Azure OpenAI Wrapper Test

This notebook tests the Azure OpenAI wrapper with mock data for meeting summarization.

## Features Tested
- Mock data generation
- Timeline-based summarization
- Azure OpenAI wrapper (mock mode)
- Full end-to-end summarization flow

## Setup

In [ ]:
import sys
sys.path.append('..')

from app.utils.mock_data_generator import MockDataGenerator
from app.services.summarization_service import SummarizationService
from app.services.azure_openai_service import AzureOpenAIService
import json
from datetime import datetime

## 1. Generate Mock Meeting Data

In [ ]:
# Generate a 10-minute mock meeting
meeting = MockDataGenerator.generate_sample_meeting(duration_minutes=10)

print(f"Meeting ID: {meeting.id}")
print(f"Duration: {meeting.duration} seconds ({meeting.duration/60:.1f} minutes)")
print(f"Participants: {meeting.participants}")
print(f"Transcript segments: {len(meeting.transcript)}")
print(f"\nFirst 3 segments:")
for i, segment in enumerate(meeting.transcript[:3]):
    print(f"  {i+1}. [{segment.start_time:.1f}s] {segment.speaker_id}: {segment.text}")

## 2. Test with MOCK OpenAI (No API Calls)

This mode doesn't require Azure OpenAI credentials and is perfect for testing.

In [ ]:
# Initialize service with mock mode
service = SummarizationService(use_mock_openai=True)

# Generate summary
summary = service.summarize_meeting(meeting)

print("=" * 80)
print("TIMELINE SUMMARIES")
print("=" * 80)

for i, ts in enumerate(summary.timeline_summaries, 1):
    print(f"\n--- Segment {i}: {ts.time_range} ---")
    print(f"Speakers: {', '.join(ts.speakers)}")
    print(f"\nTopics:")
    for topic in ts.topics:
        print(f"  • {topic}")
    print(f"\nKey Points:")
    for point in ts.key_points:
        print(f"  • {point}")
    if ts.action_items:
        print(f"\nAction Items:")
        for action in ts.action_items:
            print(f"  ✓ {action}")

print("\n" + "=" * 80)
print("OVERALL SUMMARY")
print("=" * 80)
print(f"\n{summary.overall_summary}")

print(f"\nKey Decisions:")
for decision in summary.key_decisions:
    print(f"  ✓ {decision}")

print(f"\nFollow-up Actions:")
for action in summary.follow_up_actions:
    print(f"  → {action}")

## 3. Test with Longer Meeting (Multiple Timeline Segments)

In [ ]:
# Generate a 30-minute meeting (will have multiple 5-minute segments)
long_meeting = MockDataGenerator.generate_long_meeting(duration_minutes=30)

print(f"Long Meeting:")
print(f"  Duration: {long_meeting.duration/60:.1f} minutes")
print(f"  Segments: {len(long_meeting.transcript)}")
print(f"  Participants: {long_meeting.participants}")

# Summarize
long_summary = service.summarize_meeting(long_meeting)

print(f"\nGenerated {len(long_summary.timeline_summaries)} timeline segments")
print("\nTimeline Segments:")
for i, ts in enumerate(long_summary.timeline_summaries, 1):
    print(f"  {i}. {ts.time_range}: {len(ts.topics)} topics, {len(ts.key_points)} key points")

## 4. Export Summary to JSON

In [ ]:
# Convert summary to JSON
summary_json = summary.model_dump(mode='json')

# Pretty print
print("Summary JSON:")
print(json.dumps(summary_json, indent=2, default=str))

# Save to file (optional)
# with open('../tests/mock_data/sample_summary.json', 'w') as f:
#     json.dump(summary_json, f, indent=2, default=str)

## 5. Test Azure OpenAI Service Directly

In [ ]:
# Test the Azure OpenAI service wrapper directly
openai_service = AzureOpenAIService(use_mock=True)

# Test timeline summary
test_transcript = """Speaker 1: Let's discuss the Azure integration.
Speaker 2: We need to implement Speech-to-Text with diarization.
Speaker 1: And then use OpenAI for summarization."""

result = openai_service.generate_timeline_summary(
    transcript_segment=test_transcript,
    time_range="0:00 - 5:00",
    speakers=["Speaker 1", "Speaker 2"]
)

print("Timeline Summary Result:")
print(json.dumps(result, indent=2))

## 6. Test with REAL Azure OpenAI (Requires Credentials)

⚠️ **Uncomment the code below only when you have:**
- Azure OpenAI credentials configured in `.env` file
- Set `USE_MOCK_OPENAI=false` in `.env`

This will make actual API calls to Azure OpenAI.

In [ ]:
# UNCOMMENT TO TEST WITH REAL AZURE OPENAI
# service_real = SummarizationService(use_mock_openai=False)
# summary_real = service_real.summarize_meeting(meeting)
# 
# print("Real Azure OpenAI Summary:")
# print(json.dumps(summary_real.model_dump(mode='json'), indent=2, default=str))

## 7. Save Mock Data for iOS Testing

In [ ]:
# Save sample meeting data to JSON for iOS app testing
MockDataGenerator.save_mock_data_to_json(
    filename='../tests/mock_data/sample_transcript.json',
    duration_minutes=10
)

print("Mock data saved successfully!")

## Summary

This notebook demonstrates:
1. ✅ Mock data generation for meetings
2. ✅ Timeline-based summarization (5-minute intervals)
3. ✅ Azure OpenAI wrapper with mock mode
4. ✅ Full end-to-end summarization flow
5. ✅ JSON export for API integration

**Next Steps:**
- Test with real Azure OpenAI credentials
- Integrate with FastAPI backend
- Connect iOS app to backend API
- Test with actual Azure Speech-to-Text transcripts